In [1]:
# ruff: noqa: F401, ANN201

In [2]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [3]:
import sys

from dotenv import load_dotenv

sys.path.insert(0, "..")

load_dotenv()

True

In [4]:
from datetime import datetime, timedelta
from pathlib import Path

import altair as alt
import duckdb
import hvplot.polars
import numpy as np
import polars as pl

In [5]:
from tsdb_benchmarks.settings import SETTINGS, DatabaseName, Operation, SuiteName

In [6]:
SUITE = "rtabench"

In [7]:
db = duckdb.connect(SETTINGS.results_directory / "results-replica.db")

In [8]:
db.execute(
    "select * from benchmark where finished_at is not null and deleted_at is null and suite = (?) order by started_at",
    [SUITE],
).pl()

id,suite,db,operation,started_at,finished_at,deleted_at,notes
i32,str,str,str,datetime[μs],datetime[μs],datetime[μs],str
1,"""rtabench""","""duckdb""","""populate""",2025-07-18 14:02:12.704738,2025-07-18 14:03:10.511508,null,null
2,"""rtabench""","""duckdb""","""run""",2025-07-18 14:03:10.897275,2025-07-18 14:03:16.789995,null,null
7,"""rtabench""","""clickhouse""","""populate""",2025-07-18 14:39:56.356276,2025-07-18 14:41:06.054608,null,null
8,"""rtabench""","""clickhouse""","""run""",2025-07-18 14:41:06.082501,2025-07-18 14:44:34.483136,null,null
11,"""rtabench""","""timescaledb""","""populate""",2025-07-19 10:26:27.545036,2025-07-19 10:36:00.916772,null,null
12,"""rtabench""","""timescaledb""","""run""",2025-07-19 10:36:00.977426,2025-07-19 10:38:03.479397,null,null
13,"""rtabench""","""monetdb""","""populate""",2025-07-19 10:52:41.340366,2025-07-19 11:03:25.349146,null,null
14,"""rtabench""","""monetdb""","""run""",2025-07-19 11:03:25.377888,2025-07-19 11:05:07.800880,null,null


In [9]:
df = db.execute(
    """
SELECT
	db,
	operation,
	ROUND(AVG(epoch(finished_at - started_at)), 2) AS avg_duration_seconds
FROM
	benchmark
WHERE
	suite = (?)
	AND finished_at IS NOT NULL
GROUP BY
	db,
	operation
ORDER BY
	operation,
	avg_duration_seconds;
""",
    [SUITE],
).pl()

width = 100 * df.select("db").n_unique()

chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("db:N", title="Database"),
        xOffset=alt.XOffset("operation:N"),
        y=alt.Y("avg_duration_seconds:Q", title="Avg Duration (s)"),
        color=alt.Color("operation:N", title="Operation"),
        tooltip=["db:N", "operation:N", "avg_duration_seconds:Q"],
    )
    .properties(
        width=width,
        height=400,
    )
)

chart

alt.Chart(...)

In [10]:
df = db.execute(
    """

WITH query_events AS (
    SELECT
        b.db,
        e.benchmark_id,
        e.name,
        e.time,
        e.type
    FROM event e
    JOIN benchmark b ON e.benchmark_id = b.id
    WHERE b.suite = (?)
      AND e.name ~ '^query_.*_iteration_[0-9]+$'
),
paired AS (
    SELECT
        s.db,
        REGEXP_REPLACE(s.name, '_iteration_[0-9]+$', '') AS base_query,
        CAST(REGEXP_EXTRACT(s.name, '_iteration_([0-9]+)$', 1) AS INTEGER) AS iteration,
        epoch(e.time - s.time) AS duration_seconds
    FROM query_events s
    JOIN query_events e
      ON s.db = e.db
     AND s.benchmark_id = e.benchmark_id
     AND s.name = e.name
     AND s.type = 'start'
     AND e.type = 'end'
),
aggregated AS (
    SELECT
        db,
        base_query,
        MAX(CASE WHEN iteration = 1 THEN duration_seconds END) AS first_seconds,
        AVG(CASE WHEN iteration > 1 THEN duration_seconds END) AS rest_avg_seconds,
        STDDEV_SAMP(CASE WHEN iteration > 1 THEN duration_seconds END) AS rest_stddev_seconds,
        COUNT(*) FILTER (WHERE iteration > 1) AS rest_runs
    FROM paired
    GROUP BY db, base_query
)
SELECT
	db,
	base_query as query,
	1000 * ROUND(first_seconds, 4) AS first_ms,
	1000 * ROUND(rest_avg_seconds, 4) AS rest_avg_ms,
	1000 * ROUND(rest_stddev_seconds, 4) AS rest_stddev_ms,
	rest_runs
FROM
	aggregated
ORDER BY
	query, rest_avg_ms;

""",
    [SUITE],
).pl()


df = df.with_columns(pl.col.query.str.strip_prefix("query_").str.split("_").list.first().alias("query"))


# Step 2: Identify best performers (min rest_avg_ms per query)
fastest = (
    df.group_by("query")
    .agg(pl.col("rest_avg_ms").min().alias("min_rest"))
    .join(df, on="query")
    .with_columns((pl.col("rest_avg_ms") == pl.col("min_rest")).alias("is_fastest"))
    .drop("min_rest")
)

# Step 3: Altair heatmap with conditional green border
heatmap = (
    alt.Chart(fastest)
    .mark_rect()
    .encode(
        x=alt.X("db:N", title="Database"),
        y=alt.Y("query:N", title="Query"),
        color=alt.Color(
            "rest_avg_ms:Q",
            title="Rest avg (ms)",
            scale=alt.Scale(scheme="redyellowgreen", type="log", reverse=True),
        ),
        stroke=alt.condition(
            "datum.is_fastest",
            alt.value("magenta"),
            alt.value("none"),
        ),
        strokeWidth=alt.condition(
            "datum.is_fastest",
            alt.value(3),
            alt.value(0),
        ),
        tooltip=[
            alt.Tooltip("db:N", title="Database"),
            alt.Tooltip("query:N", title="Query"),
            alt.Tooltip("rest_avg_ms:Q", title="Rest avg (ms)"),
            alt.Tooltip("rest_stddev_ms:Q", title="Rest stddev (ms)"),
            alt.Tooltip("first_ms:Q", title="First run (ms)"),
        ],
    )
    .properties(width=1000, height=600, title="Query Performance Heatmap with Fastest DB Highlighted")
)


heatmap

alt.Chart(...)

In [11]:
df = db.execute(
    """

WITH first_metric_time AS (
    SELECT
        benchmark_id,
        MIN(time) AS start_time
    FROM metric
    GROUP BY benchmark_id
)
SELECT
    b.db,
    b.suite,
    b.operation,
    epoch(m.time - fm.start_time) AS seconds_since_start,
    m.cpu_percent,
    m.mem_mb,
    m.disk_mb
FROM metric m
JOIN first_metric_time fm ON m.benchmark_id = fm.benchmark_id
JOIN benchmark b ON m.benchmark_id = b.id
WHERE b.suite = (?)
ORDER BY b.id, seconds_since_start;

""",
    [SUITE],
).pl()

df_long = df.unpivot(
    on=["cpu_percent", "mem_mb", "disk_mb"],
    index=["db", "suite", "operation", "seconds_since_start"],
    variable_name="metric",
    value_name="value",
)

df_long = df_long.with_columns(
    [
        pl.col("db").cast(pl.Categorical),
        pl.col("operation").cast(pl.Categorical),
        pl.col("metric").cast(pl.Categorical),
    ]
)

selector = alt.selection_point(fields=["db"], bind="legend")


chart = (
    alt.Chart(df_long)
    .mark_line(point=True)
    .encode(
        x=alt.X("seconds_since_start:Q"),
        y=alt.Y("value:Q"),
        color=alt.Color("db:N"),
        tooltip=[
            "db",
            "suite",
            "operation",
            "metric",
            alt.Tooltip("seconds_since_start:Q", format=".2f"),
            alt.Tooltip("value:Q", format=".1f", title="value"),
        ],
        opacity=alt.condition(selector, alt.value(1.0), alt.value(0.1)),
    )
    .add_params(selector)
    .properties(width=600, height=250)
    .facet(row=alt.Row("metric:N", title="Metric"), column=alt.Column("operation:N"))
    .configure_view(clip=True)
)

chart = chart.resolve_scale(
    y="independent",
    x="independent",
)


chart

alt.FacetChart(...)